<a href="https://colab.research.google.com/github/ShoaibSajid/Python_CNN/blob/Yolov2-Tiny/Yolov2_Tiny_Python_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content
![ ! -d "./Python_CNN" ] && echo "Github Repo DOES NOT exists."
![ ! -d "./Python_CNN" ] && git clone https://github.com/ShoaibSajid/Python_CNN
![ -d "./Python_CNN" ] && echo "Github Repo exist."
%cd Python_CNN
!git checkout Yolov2-Tiny
# %pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu116
# %pip install tqdm matplotlib mnist==0.2.2 numpy bz2file
# !pip install --upgrade --no-cache-dir gdown
![ -e "./Input_Data.pickle" ] && echo 'Default Input data already exists'
![ ! -e "./Input_Data.pickle" ] && echo 'Default Input data is missing. Downloading file.'
![ ! -e "./Input_Data.pickle" ] && gdown --id 1B8hlZbsewj4RbEVM59yce0M3Nyz54KFQ
! git pull

/content
Github Repo exist.
/content/Python_CNN
Already on 'Yolov2-Tiny'
Your branch is up to date with 'origin/Yolov2-Tiny'.
Default Input data already exists
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), 467 bytes | 155.00 KiB/s, done.
From https://github.com/ShoaibSajid/Python_CNN
   c205ef8..8835e83  Yolov2-Tiny -> origin/Yolov2-Tiny
Updating c205ef8..8835e83
Fast-forward
 cnn_python.py        | 18 +++++++++---------
 yolov2tiny_python.py | 12 ++++++------
 2 files changed, 15 insertions(+), 15 deletions(-)


In [ ]:
!pwd
!echo 
!echo List of files
!echo 
!ls

/content/Python_CNN

List of files

cnn_python.py	   Jupyter_Notebook_CNN_Python.ipynb  Temp_Files
cnn_torch.py	   LICENSE			      util
config		   loss.py			      weight_parameter
data		   Outputs			      yolov2tiny_python.py
dataset		   __pycache__			      yolov2tiny_torch.py
eecs598		   README.md
Input_Data.pickle  readme_src


In [ ]:
from pathlib import Path
import pickle
import torch
from torch.utils.data import DataLoader
from cnn_python import *

In [ ]:
python_model = DeepConvNet(input_dims=(3, 416, 416),
						num_filters=[16, 32, 64, 128, 256, 512, 1024, 1024],
						max_pools=[0, 1, 2, 3, 4],
						weight_scale='kaiming',
						batchnorm=True,
						dtype=torch.float32, device='cpu')

In [ ]:
_Load_Weights = True
if _Load_Weights:
	model = Yolov2()
	weightloader = WeightLoader()
	python_model = weightloader.load(python_model, model, './data/pretrained/yolov2-tiny-voc.weights')


In [ ]:
_Dataset = False
if _Dataset:
	from dataset.factory import get_imdb
	from dataset.roidb import RoiDataset, detection_collate
	dataset = 'voc0712trainval'
	imdb_name = 'voc_2007_trainval+voc_2012_trainval'
	imdbval_name ='voc_2007_test'

	def axpy(N: int = 0., ALPHA: float = 1., X: int = 0, INCX: int = 1, Y: float =0., INCY: int = 1):
		for i in range(N):
			Y[i * INCY] += ALPHA * X[i * INCX]

	def get_dataset(datasetnames):
		names = datasetnames.split('+')
		dataset = RoiDataset(get_imdb(names[0]))
		print('load dataset {}'.format(names[0]))
		for name in names[1:]:
			tmp = RoiDataset(get_imdb(name))
			dataset += tmp
			print('load and add dataset {}'.format(name))
		return dataset

	train_dataset = get_dataset(imdb_name)


In [ ]:
_Dataloader = _Dataset
if _Dataloader:
	train_dataloader = DataLoader(train_dataset, batch_size=64,
									shuffle=True, num_workers=2,
									collate_fn=detection_collate, drop_last=True)
	train_data_iter = iter(train_dataloader)


In [ ]:
_Get_Next_Data = _Dataloader
if _Get_Next_Data:
	_data = next(train_data_iter)
	im_data, gt_boxes, gt_classes, num_boxes = _data

	im_data     = im_data[0].unsqueeze(0)
	gt_boxes    = gt_boxes[0].unsqueeze(0)
	gt_classes  = gt_classes[0].unsqueeze(0)
	num_boxes     = num_boxes[0].unsqueeze(0)

	__data = im_data, gt_boxes, gt_classes, num_boxes

	# Path("Input").mkdir(parents=True, exist_ok=True)
	with open('Input_Data.pickle','wb') as handle:
		pickle.dump(__data,handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
	default_data = 'Input_Data.pickle'
	# if not os.path.isfile(default_data):
	# 	print('Deafult data file does not exist. Downlaoding file now.')
	# 	url = 'https://drive.google.com/uc?id=1j1zyq0lRQ_BVqSS5zM2GHU3Q4c1qH0DP'
	# 	output = default_data
	# 	gdown.download(url, output, quiet=False)
	with open(default_data, 'rb') as handle:
		b = pickle.load(handle)
	im_data, gt_boxes, gt_classes, num_boxes = b
	__data = im_data, gt_boxes, gt_classes, num_boxes
	print(f"\n\nLoading data from saved file\n\nImage (im_data[0,:3,66:69,66:69]\n{im_data[0,:3,66:69,66:69]}\n\n")
	



Loading data from saved file

Image (im_data[0,:3,66:69,66:69]
tensor([[[0.6431, 0.6431, 0.6392],
         [0.6431, 0.6431, 0.6431],
         [0.6471, 0.6471, 0.6471]],

        [[0.7373, 0.7373, 0.7333],
         [0.7373, 0.7373, 0.7412],
         [0.7412, 0.7412, 0.7373]],

        [[1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000]]])




In [ ]:
# Define flags
python_model.forward_prop 	= True  	# Perform forward propagation or load saved file.
python_model.cal_loss 		  = True    # Perform loss calculation or load save file
python_model.backward_prop 	= True  	# Perform backward propagation or load saved file
python_model.save_pickle 	  = True  	# Save output in form of pickle file
python_model.save_output 	  = True   	# Save output in form of text files

# **Method 1**

In [ ]:
# Forward, Loss and Backward
# Fout, Fcache, loss, loss_grad, BlDout, Bgrads = python_model.train(im_data, gt_boxes=gt_boxes, gt_classes=gt_classes, num_boxes=num_boxes)

# **Method 2**

In [ ]:
# Forward Propagation
X = im_data
if python_model.forward_prop:
  
  out, cache, FOut = python_model.forward(X)

  Path("Temp_Files/Python").mkdir(parents=True, exist_ok=True)
  with open('Temp_Files/Python/Forward_Out_last_layer.pickle','wb') as handle:
    pickle.dump(out,handle, protocol=pickle.HIGHEST_PROTOCOL)
  with open('Temp_Files/Python/Forward_cache.pickle','wb') as handle:
    pickle.dump(cache,handle, protocol=pickle.HIGHEST_PROTOCOL)
  with open('Temp_Files/Python/Forward_Out_all_layers.pickle','wb') as handle:
    pickle.dump(FOut,handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
  print("Loading previous files for Forward Propagation.")
  with open('Temp_Files/Python/Forward_Out_last_layer.pickle', 'rb') as handle:
    out = pickle.load(handle)
    out.requires_grad = True
    out.retain_grad()
  with open('Temp_Files/Python/Forward_Out_all_layers.pickle','rb') as handle:
    FOut = pickle.load(handle)
  with open('Temp_Files/Python/Forward_cache.pickle', 'rb') as handle:
    cache = pickle.load(handle)
   

In [ ]:
# Calculate Loss
if python_model.cal_loss:
  loss,   loss_grad = python_model.loss(out, gt_boxes=gt_boxes, gt_classes=gt_classes, num_boxes=num_boxes)
  with open('Temp_Files/Python/loss.pickle','wb') as handle:
    pickle.dump(loss,handle, protocol=pickle.HIGHEST_PROTOCOL)
  with open('Temp_Files/Python/loss_gradients.pickle','wb') as handle:
    pickle.dump(loss_grad,handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
  print("Loading previous files for Loss Calculation.")
  with open('Temp_Files/Python/loss.pickle', 'rb') as handle:
    loss = pickle.load(handle)
  with open('Temp_Files/Python/loss_gradients.pickle', 'rb') as handle:
    loss_grad = pickle.load(handle)
    

Calculating the loss and its gradients for python model.

Loss = 13.85721206665039



Loss Gradients
	torch.float32
	tensor([ 6.5163e-02, -1.5547e-02,  6.2194e-02,  1.6630e-01,  1.0815e-07,
         1.1848e-06,  2.0847e-06,  3.1208e-07,  2.2008e-06,  3.5474e-06])


In [ ]:
# Backward Propagation
if python_model.backward_prop:   
  lDout, grads = python_model.backward(loss_grad, cache)
  with open('Temp_Files/Python/Backward_lDout.pickle','wb') as handle:
    pickle.dump(lDout,handle, protocol=pickle.HIGHEST_PROTOCOL)
  with open('Temp_Files/Python/Backward_grads.pickle','wb') as handle:
    pickle.dump(grads,handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
  print("Loading previous files for Backward Propagation.")
  with open('Temp_Files/Python/Backward_lDout.pickle', 'rb') as handle:
    lDout = pickle.load(handle)
  with open('Temp_Files/Python/Backward_grads.pickle', 'rb') as handle:
    grads = pickle.load(handle)


# **Save Outputs**

In [ ]:
# Save output for circuit team.
if python_model.save_output:
  save_txt(f'Outputs/Python/Forward/Out_Last_Layer'  , out)
  save_txt(f'Outputs/Python/Forward/Out_Layer'       , FOut)
  # save_txt(f'Outputs/Python/Forward/cache_Layer'     , cache)
  save_txt(f'Outputs/Python/Loss/loss'               , loss)
  save_txt(f'Outputs/Python/Loss/loss_grad'          , loss_grad)
  save_txt(f'Outputs/Python/Backward/lDout_Layer'    , lDout)
  save_txt(f'Outputs/Python/Backward/grads'          , grads)
  save_txt(f'Outputs/Python/Parameters/'             , python_model.params)
  save_txt(f'Outputs/Python/Input_Image'             , X)
  print('Outputs have been saved')



		--> Saved Outputs/Python/Forward/Out_Last_Layer.txt

		--> Saved Outputs/Python/Forward/Out_Layer_0.txt

		--> Saved Outputs/Python/Forward/Out_Layer_1.txt

		--> Saved Outputs/Python/Forward/Out_Layer_2.txt

		--> Saved Outputs/Python/Forward/Out_Layer_3.txt

		--> Saved Outputs/Python/Forward/Out_Layer_4.txt

		--> Saved Outputs/Python/Forward/Out_Layer_5.txt

		--> Saved Outputs/Python/Forward/Out_Layer_60.txt

		--> Saved Outputs/Python/Forward/Out_Layer_61.txt

		--> Saved Outputs/Python/Forward/Out_Layer_6.txt

		--> Saved Outputs/Python/Forward/Out_Layer_7.txt

		--> Saved Outputs/Python/Forward/Out_Layer_8.txt

		--> Saved Outputs/Python/Loss/loss.txt

		--> Saved Outputs/Python/Loss/loss_grad.txt

		--> Saved Outputs/Python/Backward/lDout_Layer_8.txt

		--> Saved Outputs/Python/Backward/lDout_Layer_7.txt

		--> Saved Outputs/Python/Backward/lDout_Layer_6.txt

		--> Saved Outputs/Python/Backward/lDout_Layer_5.txt

		--> Saved Outputs/Python/Backward/lDout_Layer_4.txt

		--> 